In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
os.chdir(PROJ_ROOT)
import configparser
from configparser import ExtendedInterpolation
config_path = 'config/parameters.ini'
# read configuration from config file
pars = configparser.ConfigParser(interpolation=ExtendedInterpolation())
pars.read(config_path)

['config/parameters.ini']

In [2]:
import pickle
imputed_save_path = 'data/external/imputed_tensor_array.pkl'
with open(imputed_save_path, 'rb') as fi:
    imputed_tensor_array = pickle.load(fi)
    

In [3]:
from src.data.utils import read_query_from_file

In [5]:
import pandas as pd

In [21]:
import numpy as np

In [19]:
# start and end dates
train_start_date = [int(k) for k in pars['train_test_split']['train_start_date'].split('-')]
test_start_date = [int(k) for k in pars['train_test_split']['test_start_date'].split('-')]


In [6]:
seed_path = pars['global']['seed_query_path']
seed_word_list = read_query_from_file(seed_path)
# create placeholder df for this seed queries rank
placeholder_df = pd.DataFrame(columns=seed_word_list)


In [8]:
import ast
# read location list from pars
location_list = ast.literal_eval(pars['global']['locations'])

In [9]:
input_file_folder = '/Volumes/Samsung_T5/macPro-D/research/covid_bing_query_model_/data/external/search.csv'



In [16]:
from src.data.utils import get_location_output_path,\
    read_raw_data, process_data, select_days, read_raw_data

In [23]:
output_file_folder = 'data/processed/search.csv'

In [24]:
state_count = 0 
for location_name in location_list:
    location_train_data_path = get_location_output_path(input_file_folder, location_name)
    train_data = read_raw_data(location_train_data_path, add_datetime=True, norm_col=False, date_format='%Y-%m-%d')
    y, trend_fea = process_data(train_data, seed_word_list, placeholder_df=placeholder_df)
    trend_fea = select_days(trend_fea, train_start_date, test_start_date)
    
    # state save path 
    location_save_data_path = get_location_output_path(output_file_folder, location_name)
    # fill nas 
    trend_fea_array = trend_fea.fillna(np.nan)
    trend_fea_array = np.array(trend_fea_array).T
    na_pos = np.where(np.isnan(trend_fea_array))
    
    # fill na 
    city_imputed_array = imputed_tensor_array[state_count]
    
    trend_fea_array[na_pos] = city_imputed_array[na_pos]
    city_imputed_pd = pd.DataFrame(columns = trend_fea.columns, data = trend_fea_array.T)
    city_imputed_pd.index = trend_fea.index
    city_imputed_pd.to_csv(location_save_data_path, header = True, index = True)
    state_count += 1    